## Transfer Learning

In this notebook, we will train a convolutional neural network using the concept of transfer learning. To this end, we will start with a CNN model pre-trained on ImageNet and fine-tune it for our specific task.

Keras Applications are deep learning models that are made available alongside pre-trained weights. These models can be used for prediction, feature extraction, and fine-tuning.

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K


## Load data

In [11]:
import numpy as np
import os

list_not_dir = ['gdc_manifest_20170512_072412.txt', 'test.svs', 'y_train.txt', 'gdc-client']
y_train = {}
with open("data/y_train.txt") as f:
    for line in f:
        (key, val) = line.split()
        key = str(key).replace('.','-')
        y_train[key] = int(val)
        
svs_images = []
for filename in os.listdir("/home/cedoz/data/"):
    if filename not in list_not_dir:
        svs_images.append(filename[:-4])
        
intersect_samples = [val for val in svs_images if val in y_train.keys()]


In [15]:
#Filter y_train to contain only the common images
y_train = {key: y_train[key] for key in intersect_samples}
svs_images = intersect_samples

## FC layers fine tuning

In [ ]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a sigmoid layer -- we have 2 classes: hyper and hypo
predictions = Dense(1, activation='sigmoid')(x)
# if we had more classes:
# predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples: x and y are np arrays
model.fit(x, y, batch_size=32, epochs=10, verbose=1, callbacks=None, validation_data=None, shuffle=True, class_weight=None, sample_weight=None, initial_epoch=0)

# If we can't load all the data in memory, we will need to use a data generator for each batch
# model.fit_generator(self, generator, steps_per_epoch, epochs=1, verbose=1,callbacks=None, validation_data=None,
#                    validation_steps=None, class_weight=None, max_q_size=10, workers=1, pickle_safe=False, initial_epoch=0)


## Conv Layers Fine Tuning

At this point, the top layers are well trained and we can start fine-tuning convolutional layers from inception V3. We will freeze the bottom N layers and train the remaining top layers.

In [ ]:
# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)